In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint

import io
import sys
sys.path.append('../../')

from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn

from architectures import build_net, smallnet3

wandb.login()


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [2]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [3]:
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.5, 0.4, 0.3, 0.2, 0.7]
        },
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 120
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[1069888]
        },
        'optimizer': {
            'values': ['adam']
        },
        'scheduler': {
            'values': [0.2, 0.3, 0.1, 0.5]
        },
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 5e-5,
            'max': 1e-4
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
        'data_set':{
            'values':['Augmented']
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels': {
            'valaues': [3]
        },
        'model_name': {
            'values':['smallnet2']
        },
        'num_classes': {
            'values': [11]
        },
         'model_name' : {'values': ['smallnet2']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/optics/AugmentedDS_IDSW/']}
        },
    }
    
title = f'IDSW_unwrap_3c2l_112023'

save_dict = {'Run' : title,
            'Current_Epoch': 0}

col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}


sweep_id = wandb.sweep(config, project=f"112023_3conv2lin_IDSW_{col_dict['colour']}_{col_dict['size']}")

Create sweep with ID: 3ye6lngb
Sweep URL: https://wandb.ai/antvis/112023_3conv2lin_IDSW_colour_%5B452%2C%20144%5D/sweeps/3ye6lngb


In [4]:
pprint.pprint(config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'channels': {'values': [3]},
                'data_set': {'values': ['Augmented']},
                'dropout': {'values': [0.5, 0.4, 0.3, 0.2, 0.7]},
                'epochs': {'value': 120},
                'first_lin_lay': {'values': [1069888]},
                'image_path': {'values': ['/its/home/nn268/antvis/optics/AugmentedDS_IDSW/']},
                'ks': {'values': [(3, 5)]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.0001,
                                  'min': 5e-05},
                'lin_layer_size': {'values': [100]},
                'loss_fn': {'values': ['MSE', 'CrossEntropy']},
                'model_name': {'values': ['smallnet2']},
                'num_classes': {'values': [11]},
                'optimizer': {'values': ['adam']},
                'scheduler': {'values': [0.2, 0.3, 0.1, 0.5]},
                

In [ ]:
def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = hp_sweep(config, col_dict, save_dict)

wandb.agent(sweep_id, tr, count=25)

wandb: Agent Starting Run: 1uqh45g1 with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.3
wandb: 	epochs: 120
wandb: 	first_lin_lay: 1069888
wandb: 	image_path: /its/home/nn268/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 7.926280920896969e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: MSE
wandb: 	model_name: smallnet2
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.3
wandb: 	weight_decay: 2e-05


/its/home/nn268/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


improvment in metrics. model saved


In [ ]:
"""
config = dict(
    epochs= 80, 
    learning_rate =3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay =4096,
    lin_layer_size= 100,
    ks = 3,
    in_chan = 3
)



col_dict = {
    'colour': 'colour',
    'size': [32,32]
}"""